In [1]:
import openai
from sentence_transformers import SentenceTransformer
import datasets, accelerate
import torch
import torch.nn.functional as F
import os

device ="cuda" if torch.cuda.is_available() else "cpu"

e:\Code\Project\Law_Assistant\.venv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [24]:
f = open(".key")
org = f.readline().strip()
api_key = f.readline().strip()

In [25]:
from openai import OpenAI

client = OpenAI(
    organization=org,
    api_key = api_key
)

print(org)
print(api_key)

org-RGQs6pctYkiwR9b9jSv3rGkq
sk-KIQAwBuf2P83cUWc8KwnL_qYAuIa2KvFSRna7Ml9neT3BlbkFJ-3N3foxvMiBvoiQ36tOvuzHGul33ZFtd8RyT91S6QA


In [26]:
model = SentenceTransformer(
    "checkpoint-204", device=device, truncate_dim = 64
)


In [27]:
all_laws = []

files = sorted(os.listdir("preprocessed_laws"))

for i,file in enumerate(files):
    if((i+1) % 1000 == 0):
        print(i+1)
    f = open("preprocessed_laws/" + file, 'r', encoding="utf-8")
    l = f.read()
    all_laws.append(l[:3500])


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000


In [28]:
print(len(all_laws))
print(all_laws[0])

24721
[01_2011_QH13] 

LUẬT LƯU TRỮ
Điều 1. Phạm vi điều chỉnh và đối tượng áp dụng

Khoản 1, 2:

1. Luật này quy định về hoạt động lưu trữ; quyền và nghĩa vụ của cơ quan, tổ chức, cá nhân trong hoạt động lưu trữ; đào tạo, bồi dưỡng nghiệp vụ lưu trữ; hoạt động dịch vụ lưu trữ và quản lý về lưu trữ.
2. Luật này áp dụng đối với cơ quan nhà nước, tổ chức chính trị, tổ chức chính trị - xã hội, tổ chức chính trị xã hội - nghề nghiệp, tổ chức xã hội, tổ chức xã hội - nghề nghiệp, tổ chức kinh tế, đơn vị sự nghiệp, đơn vị vũ trang nhân dân (sau đây gọi chung là cơ quan, tổ chức) và cá nhân.



In [7]:
vector_db = torch.tensor(torch.load('embedding_64.pt'), dtype=float).to(device)
print(vector_db.shape)


torch.Size([24721, 1, 64])


C:\Users\Trung\AppData\Local\Temp\ipykernel_5756\3830621532.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vector_db = torch.tensor(torch.load('embedding_64.pt'), dtype

In [8]:
def fine_related_doc(question, n = 3):
    embedding = torch.tensor([model.encode(question)], dtype = float).to(device)
    
    max_sim = []

    for i,v in enumerate(vector_db):
        if((i+1) % 5000 == 0):
            print(f"{i+1}/{vector_db.shape[0]}")
        sim = F.cosine_similarity(embedding, v)
        
        if(sim >= 0.6):
            max_sim.append((sim,i))

    print("Process Done")
    print("Finding Related Docs...")

    max_sim = sorted(max_sim, key=lambda x: x[0], reverse=True)

    docs = []
    for i in max_sim[:n]:
        print(i)
        docs.append(all_laws[i[1]])
    return docs


In [9]:
q = """

    Quyền trẻ em là gì ?
    
"""

In [10]:
related_docs = fine_related_doc(q)

prompt = "Tôi cung cấp cho bạn một số văn bản pháp luật có liên quan như sau: \n\n"

for doc in related_docs:
    prompt += doc + "\n\n"

prompt += f"Bạn sẽ là người từ vấn pháp luật cho tôi. Từ những văn bản pháp luật trên, hãy giúp tôi trả lời câu hỏi sau: \n\n {q} \n CHÚ Ý: Nếu các văn bản không liên quan hoặc không cung cấp đủ thông tin thì hãy từ chối trả lời câu hỏi."



C:\Users\Trung\AppData\Local\Temp\ipykernel_5756\1961435994.py:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  embedding = torch.tensor([model.encode(question)], dtype = float).to(device)


5000/24721
10000/24721
15000/24721
20000/24721
Process Done
Finding Related Docs...
(tensor([0.7332], device='cuda:0', dtype=torch.float64), 4096)
(tensor([0.7197], device='cuda:0', dtype=torch.float64), 4115)
(tensor([0.7106], device='cuda:0', dtype=torch.float64), 4099)


In [11]:
for doc in related_docs:
    print(doc)


[102_2016_QH13] 

LUẬT TRẺ EM
Điều 4. Giải thích từ ngữ

Khoản 1, 2, 3:
Trong Luật này, các từ ngữ dưới đây được hiểu như sau:

1. Bảo vệ trẻ em là việc thực hiện các biện pháp phù hợp để bảo đảm trẻ em được sống an toàn, lành mạnh; phòng ngừa, ngăn chặn và xử lý các hành vi xâm hại trẻ em; trợ giúp trẻ em có hoàn cảnh đặc biệt.
2. Phát triển toàn diện của trẻ em là sự phát triển đồng thời cả về thể chất, trí tuệ, tinh thần, đạo đức và mối quan hệ xã hội của trẻ em.
3. Chăm sóc thay thế là việc tổ chức, gia đình, cá nhân nhận trẻ em về chăm sóc, nuôi dưỡng khi trẻ em không còn cha mẹ; trẻ em không được hoặc không thể sống cùng cha đẻ, mẹ đẻ; trẻ em bị ảnh hưởng bởi thiên tai, thảm họa, xung đột vũ trang nhằm bảo đảm sự an toàn và lợi ích tốt nhất của trẻ em.

[102_2016_QH13] 

LUẬT TRẺ EM
Điều 15. Quyền được chăm sóc, nuôi dưỡng
Trẻ em có quyền được chăm sóc, nuôi dưỡng để phát triển toàn diện.

[102_2016_QH13] 

LUẬT TRẺ EM
Điều 4. Giải thích từ ngữ

Khoản 10, 11:

10. Trẻ em có hoàn 

In [30]:


stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": f"{prompt}"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

Từ các văn bản pháp luật bạn cung cấp, quyền trẻ em có thể hiểu như sau:

1. **Quyền được chăm sóc, nuôi dưỡng**: Theo Điều 15 của Luật Trẻ em, trẻ em có quyền được chăm sóc, nuôi dưỡng để phát triển toàn diện.

2. **Quyền sống và được bảo vệ**: Điều 4, Khoản 10 định nghĩa trẻ em có hoàn cảnh đặc biệt là những trẻ em không đủ điều kiện thực hiện quyền sống và quyền được bảo vệ. Điều này ngầm hiểu rằng quyền sống và quyền được bảo vệ là những quyền cơ bản của trẻ em.

3. **Quyền phát triển toàn diện**: Điều 4, Khoản 2 nêu rõ phát triển toàn diện của trẻ em bao gồm sự phát triển về thể chất, trí tuệ, tinh thần, đạo đức và mối quan hệ xã hội.

4. **Giám sát việc thực hiện quyền trẻ em**: Điều 4, Khoản 11 đề cập đến việc giám sát thực hiện quyền trẻ em, cho thấy rằng trẻ em có quyền được lắng nghe ý kiến và nguyện vọng của mình trong các vấn đề liên quan đến cuộc sống và sự phát triển.

Tóm lại, quyền trẻ em là những quyền cơ bản liên quan đến sự tồn tại, sự phát triển toàn diện và sự bảo 